# Задание 1

In [1]:
import pandas as pd
import numpy as np
from statistics import median

%load_ext lab_black

In [2]:
# загрузим данные
df_center = pd.read_csv("/home/sk27/repo/eremeev/test_projects/call_center_data.csv")

In [3]:
# преобразуем формат данных о дате и времени совершения операции
df_center["datetime"] = df_center["date"] + " " + df_center["time"]
df_center["datetime"] = df_center["datetime"].astype("datetime64[ns]")
df_center.drop(["date", "time"], axis=1, inplace=True)

In [4]:
# создадим пустой словать, в котором будем собирать итоговые значения: действие (ключ) - медианная продолжительность (значение)
dict_action = {}

# внешний цикл для каждого уникального действия:
for act in list(df_center["action"].unique()):

    # создадим пустой список, куда будут помещаться медианные значения продолжительности действий по каждому юзеру
    list_time_action = []

    # внутренний цикл для каждого уникального пользователя:
    for u_id in list(df_center[(df_center["action"] == act)]["user_id"].unique()):

        # создадим пустой список, куда будут помещаться интервалы (в секундах) между последовательными действиями
        list_time = []

        # фильтруем датафрейм, осталяя только данные рассматриваемого действия и рассматриваемого пользователя
        df_test = df_center[
            (df_center.action == act) & (df_center.user_id == u_id)
        ].reset_index(drop=True)

        # пройдём по датафрейму, рассчитывая интервал времени между двумя последовательными действиями
        # и добавляя результат в список, если величина интервала не превышает 1200 сек (20 мин)
        for _ in range(1, df_test.shape[0] - 1):
            delta = df_test.iloc[_].datetime - df_test.iloc[_ - 1].datetime
            if delta / np.timedelta64(1, "s") <= 1200:
                list_time.append(delta / np.timedelta64(1, "s"))

        # добавляем в список медианных значений продолжительности выполнения действия каждым юзером.
        # обработка исключения добавлена, поскольку в отдельных случаях между всеми или почти всеми
        # действиями юзера проходит более 20 мин и код падает с ошибкой, не умея определить медианное
        # значение для пустого списка
        try:
            list_time_action.append(median(list_time))

        # если конкретный юзер при осуществлении конкретного действия допускал только интервалы продолжительностью
        # свыше 20 минут, игнорируем данные
        except:
            pass

    # занесём в словарь пару действие - медианное значение продолжительности
    # про обработку исключений - аналогично сказанному в предыдущих комментариях
    try:
        dict_action[act] = median(list_time_action)
    except:
        pass

# в результате медианные значения продолжительности получены для 40 действий, ещё для 3 действий её значение, по-видимому, превышает ограниение в 20 минут

In [5]:
# оформим результат согласно ТЗ
df_result = pd.DataFrame(list(dict_action.items()), columns=["action", "md"])
df_result["median_duration"] = pd.to_datetime(df_result["md"], unit="m").dt.time
df_result.drop(["md"], axis=1, inplace=True)

In [6]:
df_result

,action,median_duration
0,2006,00:41:30
1,2001,01:05:00
2,14,00:28:30
3,9001,01:51:00
4,2002,00:11:00
5,15,00:12:00
6,2003,00:22:30
7,7,00:51:00
8,27,00:11:00
9,758,00:01:00


***